In [138]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import re

In [102]:
soup = BeautifulSoup(requests.get("https://southpark.fandom.com/wiki/Portal:Scripts").text)

In [111]:
BASE_URL = "https://southpark.fandom.com"
season_urls = soup.find('div', id='gallery-0').find_all("a")
urls = {}
for tag in season_urls:
    urls[tag["href"]] = None
season_urls = list(map(lambda key: BASE_URL + key, urls.keys()))
season_urls

['https://southpark.fandom.com/wiki/Portal:Scripts/Season_One',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Two',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Three',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Four',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Five',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Six',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Seven',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Eight',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Nine',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Ten',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Eleven',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Twelve',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Thirteen',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Fourteen',
 'https://southpark.fandom.com/wiki/Portal:Scripts/Season_Fifteen',
 'https://s

In [126]:
def get_episode_urls(season_url: url):
    season_data = BeautifulSoup(requests.get(season_url).text)
    episode_overview = season_data.find('div', id='gallery-0').find_all("a")

    episode_urls = {}
    for episode in episode_overview:
        episode_urls[episode["href"]] = None
    return list(map(lambda key: BASE_URL + key, episode_urls.keys()))

In [134]:
def get_episode_script(episode_url: str, season: int, episode: int):
    """ Given the full url (base + episode url), scrape the script and return title + lines """
    episode_data = requests.get(episode_url).text
    script_rows = BeautifulSoup(episode_data).find_all("table", class_="headerscontent")[-1].find_all("tr", style="")
    # get title
    episode_title = script_rows[0].find("td").text.strip()
    print(f"Episode title: {episode_title}")
    # get script
    script_rows = script_rows[1:]
    lines = []
    for row in script_rows:
        line = row.find_all("td")
        actor = line[0].text.strip()
        dialogue = line[1].text.strip()
        lines.append([season, episode, actor, dialogue])
    return episode_title, lines

In [136]:
def slugify(text: str):
    # Replace non-alphanumeric characters (except hyphens and underscores) with hyphens
    slug = re.sub(r'[^a-zA-Z0-9-_]', '-', text)
    
    # Replace consecutive hyphens with a single hyphen
    slug = re.sub(r'-+', '-', slug)
    
    # Remove leading and trailing hyphens
    slug = slug.strip('-')
    
    # Convert to lowercase
    slug = slug.lower()
    
    return slug

In [141]:
SEASON = 1
for season_url in season_urls:
    episode_urls = get_episode_urls(season_url)
    EPISODE = 1
    for episode_url in episode_urls:
        episode_title, episode_lines = get_episode_script(episode_url, SEASON, EPISODE)
        episode_title = slugify(episode_title)
        df = pd.DataFrame(episode_lines)
        dir = os.path.join(".", "episodes_csv", f"s{SEASON}")
        if not os.path.exists(dir):
            os.makedirs(dir)
        # Save the DataFrame to a CSV file with a custom delimiter (semicolon in this case)
        df.to_csv(os.path.join(dir, f"e{EPISODE}_{episode_title}.csv"), sep=';', index=False, header=False)
        EPISODE += 1
    SEASON += 1

Episode title: Cartman Gets an Anal Probe
Episode title: Weight Gain 4000
Episode title: Volcano
Episode title: Big Gay Al's Big Gay Boat Ride
Episode title: An Elephant Makes Love to a Pig
Episode title: Death
Episode title: Pinkeye
Episode title: Starvin' Marvin
Episode title: Mr. Hankey, the Christmas Poo
Episode title: Damien
Episode title: Tom's Rhinoplasty
Episode title: Mecha-Streisand
Episode title: Cartman's Mom is a Dirty Slut
Episode title: Terrance and Phillip in Not Without My Anus
Episode title: Cartman's Mom is Still a Dirty Slut
Episode title: Chickenlover
Episode title: Ike's Wee Wee
Episode title: Conjoined Fetus Lady
Episode title: The Mexican Staring Frog of Southern Sri Lanka
Episode title: City on the Edge of Forever (Flashbacks)
Episode title: Summer Sucks
Episode title: Chef's Chocolate Salty Balls
Episode title: Chickenpox
Episode title: Roger Ebert Should Lay off the Fatty Foods
Episode title: Clubhouses
Episode title: Cow Days
Episode title: Chef Aid
Episode 